In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ.get('riot_api_key')


In [2]:
import pygsheets 
google_url = os.environ.get('google_url')
service_account = pygsheets.authorize(service_account_file = 'JSONS\\spreadsheet-automator-447604-0758cb3da318.json')
sheet = service_account.open_by_url(google_url)

In [3]:
import pandas as pd 
import requests
import time
def getLadder(rank,division):
    divisions = ['II','III','IV']
    page = pd.DataFrame()
    root = 'https://na1.api.riotgames.com/'
    meta = f'lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page=1'
    api_url = root + meta + '&api_key=' + api_key
    api_response = requests.get(api_url)
    api_df = pd.DataFrame(api_response.json())
    page = pd.concat([page,api_df])
    page_num = 2 
    while not api_df.empty:
        meta = f'lol/league/v4/entries/RANKED_SOLO_5x5/{rank}/{division}?page={page_num}'
        api_url = root + meta + '&api_key=' + api_key
        api_response = requests.get(api_url)
        api_df = pd.DataFrame(api_response.json())
        page = pd.concat([page,api_df])
        page_num+=1
        time.sleep(1)
    return page

In [4]:
def get_puuid(summoner_id):
    root = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/'
    api_url = root + summoner_id + '?api_key=' + api_key
    page = requests.get(api_url)
    return page.json()['puuid']

In [25]:
if os.path.exists('ironLadder.csv'):
    ironLadder = pd.read_csv('ironLadder.csv')
else:
    ironLadder = getLadder('IRON','I').sort_values('leaguePoints',ascending=False).reset_index(drop=True)
    ironLadder.to_csv('ironLadder.csv',index=False)
    print('Not found')

In [18]:
def getMatch(matchID):
    root = 'https://americas.api.riotgames.com/lol/match/v5/matches/'
    middle = '?api_key='
    api_url = root + matchID + middle + api_key
    time.sleep(1)
    return requests.get(api_url).json()

In [16]:
def getListMatches(puuid,start,end):
    root = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/'
    middle = '/ids?type=ranked&'
    part = 'start='+str(start)+'&count='+str(end)
    api_url = root +str(puuid)+middle + part + '&api_key=' + api_key
    page = requests.get(api_url).json()
    time.sleep(1)
    return page 

In [ ]:
def getKDA(information):
    kda = 0
    return kda

In [ ]:
def getCS(information):
    cs = 0
    return cs

In [30]:
# For each person in the ladder, look at their 0 - 10 games and get their average cs, KDA
average_iron_1 = {'puuid':[],'KDA':[],'CS':[]}
for i in ironLadder.itertuples():
    puuid = get_puuid(i.summonerId)
    matches = getListMatches(puuid=puuid,start = 10,end = 20)
    kda_mean = 0 
    cs_mean = 0 
    for m in matches:
        information = getMatch(m)
        kda = getKDA(information)
        cs = getCS(information)
        kda_mean += kda
        cs_mean += cs
        time.sleep(5)
    kda_mean= kda_mean/10
    cs_mean = cs_mean/10
    new_row = pd.DataFrame({'puuid':puuid,'KDA':kda_mean,'CS':cs_mean})
    average_iron_1 = pd.concat([average_iron_1,new_row],ignore_index=True)

SyntaxError: incomplete input (995184217.py, line 4)